In [26]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from config import DATABASE_URI

plt.rcParams["figure.figsize"] = (15,10)

In [2]:
tickers = ["BLV", "EMB", "HYG", "VEU", "VTI"]
with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT date, ticker, closeadj FROM prices WHERE ticker IN {tuple(tickers)}")
        results = cur.fetchall()

In [3]:
df = pd.DataFrame.from_records(results, columns=["date", "ticker", "closeadj"], coerce_float=True)
# Set index, sort index, then transform into Series via squeeze
df = df.set_index(["date", "ticker"], verify_integrity=True).sort_index().squeeze()

In [5]:
returns = df.unstack().pct_change().dropna()

In [113]:
mean = np.mean(returns, axis=0)
covariance = np.cov(returns.T)

In [114]:
def minvar(x, covar):
    return x.T @ covar @ x

In [148]:
initial = np.ones(len(returns.columns)) / len(returns.columns)
constraints = [{"type": "eq", "fun": lambda x: np.sum(x) - 1}]
bounds = [(0, 1) for x in range(len(returns.columns))]

results = minimize(minvar, initial, args=(covariance), constraints=constraints, bounds=bounds, tol=1e-15)

In [149]:
for i, x in enumerate(results.x):
    print(f"{returns.columns[i]}: {round(x * 100, 2)}%")

BLV: 44.69%
EMB: 21.78%
HYG: 30.98%
VEU: 0.0%
VTI: 2.55%
